<a href="https://colab.research.google.com/github/edward0210/test/blob/main/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from keras.datasets import cifar10
import numpy as np
from keras.utils import np_utils
import tensorflow as tf
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from PIL import Image
from keras.callbacks import EarlyStopping

In [15]:
(X_train, y_train) , (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype(float)/255.0
y_train = np_utils.to_categorical(y_train)

X_val = X_train[40000:]
y_val = y_train[:40000]

X_train = X_train[:40000]
y_train = y_train[:40000]

X_test = X_test.astype(float)/255.0
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]


In [4]:
print(num_classes)

10


In [14]:
###Model

#X = tf.keras.layers.InputLayer(shape=[32,32,3])
X = tf.keras.layers.Input(shape = [32,32,3])
H = tf.keras.layers.Convolution2D(32,kernel_size = 3,strides =(1,1),padding ='same',activation = 'leaky_relu')(X)
H = tf.keras.layers.MaxPooling2D()(H)
print(H.shape)
H = tf.keras.layers.Convolution2D(16,kernel_size =3, strides = (1,1), padding = 'same',activation = 'leaky_relu')(H)
H = tf.keras.layers.MaxPooling2D()(H)
H = tf.keras.layers.Convolution2D(8,kernel_size =3, strides = (1,1), padding = 'same',activation = 'leaky_relu')(H)
H = tf.keras.layers.MaxPooling2D()(H)
H = tf.keras.layers.Flatten()(H)
H = tf.keras.layers.Dense(120,activation = 'swish',name ='fc1')(H) 
H = tf.keras.layers.Dense(60,activation = 'swish',name ='fc2')(H)
Y = tf.keras.layers.Dense(10,activation = 'softmax',name = 'fc3')(H)


model = tf.keras.models.Model(X,Y)

model.compile(loss='categorical_crossentropy',metrics='accuracy')


(None, 16, 16, 32)


In [13]:
early_stopping = EarlyStopping(monitor = 'val_loss',patience=50, verbose =1, mode= 'auto', restore_best_weights=True)

In [ ]:
hist = model.fit(X_train, y_train, epochs=10, callbacks=[early_stopping],batch_size = 4000, validation_data =(X_val,y_val))

Epoch 1/10
 5/10 [==============>...............] - ETA: 35s - loss: 2.2884 - accuracy: 0.1175

In [ ]:
plt.plot(hist.history['accuracy'],'r')
plt.plot(hist.history['loss'],'g')

pred = model.predict(X_test)
result = model.evaluate(X_test,y_test,batch_size=128)
#result = pd.DataFrame(pred).round(2)

print('test lossm test acc',result)

plt.show()

In [ ]:
feat_extractor = tf.keras.models.Model(inputs=model.input,outputs=model.get_layer('fc2').output)

In [ ]:
features = feat_extractor.predict(X_train)

In [ ]:

tsne = TSNE().fit_transform(features)
tx, ty = tsne[:,0], tsne[:,1]
tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))

In [ ]:

width = 3000
height =3000
max_dim = 100
full_image = Image.new('RGB', (width, height))
#print(X_test)
for idx, x in enumerate(X_test):

    tile = Image.fromarray(np.uint8(x * 255))
    rs = max(1, tile.width / max_dim, tile.height / max_dim)
    tile = tile.resize((int(tile.width / rs),
                        int(tile.height / rs)),
                       Image.ANTIALIAS)
    full_image.paste(tile, (int((width-max_dim) * tx[idx]),
                            int((height-max_dim) * ty[idx])))

In [ ]:
plt.imshow(full_image)

In [ ]:
#full_image = full_image[:,:,::-1]
print(full_image.format)
#plt.imsave('d4.bmp',full_image)
full_image.save('d6.jpg')